
### Main Sections:

1. TensorFlow Basics:
    * Introduction to TensorFlow
    * Installing and Importing TensorFlow
    * Constants and Variables
    * Simple Operations and Tensors
    * Placeholders and Data Feeding
2. Neural Network Fundamentals:
    * Building a Linear Regression Model
    * Loss Functions and Optimization
    * Activation Functions
    * Training Loop and Metrics
3. Hyperparameter Tuning:
    * Importance of Hyperparameters
    * Common Hyperparameters (batch size, epoch, learning rate)
    * GridSearchCV and RandomSearch for Exploration
    * Visualizing Tuning Results
4. Deep Learning Layers:
    * Dense Layers: Parameters and Activation Functions
    * Dropout Layers for Regularization
    * Additional Layer Types (convolutional, recurrent, pooling)
    * Compile Parameters and Training Configuration
5. Additional Notes:
    * Code snippets and resources for further learning
    * Specific questions or challenges you encountered

### Sub-Sections (optional):
    * Within each main section, you can create sub-sections for specific topics or examples.
    * Consider adding headers, bullet points, and diagrams for better organization and clarity.
    * Use different colors or fonts to highlight important information.



### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# special imports for CV and NLP
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)
# 2.7.0

### Helping Variables

In [ ]:
NUM_EPOCHS = 2
BATCH_SIZE = 32
SEED = 1

IMG_HEIGHT = 300
IMG_WIDTH = 300
CHANNELS = 3

num_inputs = 1
num_classes = 3

X_train = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0], dtype=float)
y_train = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.5, 5.0, 5.5], dtype=float)

X_test = np.array([7.0, 11.0], dtype=float)
y_test = np.array([4.0, 6.0], dtype=float)

### Get time series data

In [ ]:
# get some time series data
# the original code is taken from:
# https://github.com/https-deeplearning-ai/tensorflow-1-public

def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)


def trend(time, slope=0):
    return slope * time


def seasonal_pattern(season_time):
    """ Just an arbitrary pattern """
    return np.where(season_time < 0.3,
                    np.sin(season_time * 4 * np.pi),
                    1 / np.exp(7 * season_time))


def seasonality(time, period, amplitude=1, phase=0):
    """ Repeats the same pattern at each period """
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level


time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, 0.1)
baseline = 10
amplitude = 40
slope = 0.01
noise_level = 2

# Create the series
series = baseline + trend(time, slope) + \
    seasonality(time, period=365, amplitude=amplitude)
# Update with noise
series += noise(time, noise_level, seed=42)

split_time = 3000
time_train = time[:split_time]
series_train = series[:split_time]

WINDOW_SIZE = 20
SHUFFLE_BUFFER_SIZE = 1000

plot_series(time, series)

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(
        lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

dataset = windowed_dataset(series_train,
                           WINDOW_SIZE,
                           BATCH_SIZE,
                           SHUFFLE_BUFFER_SIZE)

### ImageDataGenerator

Official documentation - https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
# extract Horse or Human datasets for computer vision (ImageDataGenerator)
# downloaded from: https://laurencemoroney.com/datasets.html

import os
import zipfile
import shutil

# Get the Horse or Human dataset
path_horse_or_human = "data/horse-or-human.zip"
# Get the Horse or Human Validation dataset
path_validation_horse_or_human = "data/validation-horse-or-human.zip"

local_zip = path_horse_or_human
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/horse-or-human')
zip_ref.close()

local_zip = path_validation_horse_or_human
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

train_dir = 'data/horse-or-human'
validation_dir = 'data/validation-horse-or-human'

In [ ]:
# augmented ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# default ImageDataGenerator without augmentation
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # 'categorical'
)

Found 1027 images belonging to 2 classes.

Found 256 images belonging to 2 classes.

In [ ]:
 # Another option is not to specify the validation set explicitly,
# but to entrust the split to ImageDataGenerator
# In this case, be sure that the seed parameter is the same
# for both sets, otherwise the sets may overlap

train_datagen = ImageDataGenerator(rescale=1./255,
                                   # set validation split ratio
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # 'categorical'
    subset='training',  # set as training data
    seed=SEED)

validation_generator = train_datagen.flow_from_directory(
    train_dir,  # the same directory
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # 'categorical'
    subset='validation',  # set as validation data
    seed=SEED)

Found 822 images belonging to 2 classes.

Found 205 images belonging to 2 classes.

### Tokenize and pad sentences


Official documentation for Tokenizer - https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/TokenizerOfficial documentation for pad_sequences method - https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

### Nerual Netowrk

A neural network model is a collection of layers.

A layer is a defined set of computations which takes a given tensor as input and produces another tensor as output.

For example, a simple layer could just add 1 to all the elements of an input tensor.

The important point is that a layer manipulates (performs a mathematical operation on) an input tensor in some way to produce an output tensor.

Combine a series of layers together and you have a model.

The term “deep learning” comes from the stacking of large numbers of layers on top of eachother (deep in this sense is a synonym for large).

The best way to stack layers together to find patterns in data is constantly changing.

This is why techniques such as *transfer learning* are helpful because they allow you to leverage what has worked for someone else’s similar problem and tailor it to your own.

### Layers
create layers using the `tf.keras.layers` module.

Example include [Dense(fully connected)] and [Conv2D(convolutional)] layes:

In [ ]:
import tensorflow as tf
# Create a dense (or fully connected) layer with TensorFlow
dense_layer = tf.keras.layers.Dense(units,  activation=tf.keras.activations.relu)

# Create a 2D convolutional layer with TensorFlow
conv2d_layer = tf.keras.layers.Conv2D(filters, kernel_size, activation=tf.keras.activations.relu)

And there are many more pre-built layer types available in the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers).

### Neural Network Models in TensorFlow
The most straightforward way in TensorFlow to a neural network model us using the `tf.keras.Sequential` API, which allows you to stack layers in a way that the computation will be performed sequentially (one layer after the other):

In [ ]:
# Create a basic neural network model with TensorFlow
model = models.Sequential()

#
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape),
    # Add layers here
		# Computation will happen layer by layer sequentially
])

In [ ]:
from tensorflow.keras import datasets, layers, models
# Creating the Layers


# for Neural Network Model
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(512, activation=tf.nn.relu))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation=tf.nn.softmax))

 1. Typical Neural Network Architectures using Sequential API

Official documentation - https://www.tensorflow.org/guide/keras/sequential_model

In [ ]:
 # DNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu',
                          input_shape=(num_inputs, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='relu'),  # regression
])

model.summary()

In [ ]:
 # CNN
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu',
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # binary classification
    # multi-class classification
    # layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
 # RNN for NLP
model = tf.keras.Sequential([
    layers.Embedding(vocab_size,
                     embedding_dim,
                     input_length=max_length),
    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(16)),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(6, activation='softmax')  # multiclass classification
])

model.summary()

In [ ]:
# RNN for time series
model = tf.keras.Sequential([
    layers.Conv1D(filters=128, kernel_size=7,
                  strides=1, padding="causal",
                  activation="relu",
                  input_shape=(WINDOW_SIZE, 1)),
    # univariate time series - predict a value based on
    # 'WINDOW_SIZE' previous steps for 1 feature

    layers.LSTM(32, return_sequences=True),
    layers.LSTM(16, return_sequences=True),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)  # predict one value
])

model.summary()

### Compile Model
Before we start training the model on data we’ve got to compile it model with an optimizer, loss function, and metric.

All three of these are customizable depending on the project you’re working on.

- **Loss function** = measures how wrong the model is (the higher the loss, the more wrong the model, so lower is better). Common loss values include `tf.keras.losses.CategoricalCrossentropy` for multi-class classification problems and `tf.keras.losses.BinaryCrossentropy` for binary classification problems.

- **Optimizer** = tries to adjust the models parameters to lower the loss value. Common optimizers include `tf.keras.optimizers.SGD` (Stochastic Gradient Descent) and `tf.keras.optimizers.Adam`. Each optimizer comes with generally good default settings, however, of the most important values to set is the learning_rate hyperparameter.

- **Metric** = human readable value to interpret how the model is going. Metrics can be defined via `tf.keras.metrics` such as `tf.keras.metrics.Accuracy` or via a list of strings such as `['accuracy']`.

In [ ]:
# Compile model before training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

### Train Model
You can train your TensorFlow models using the `tf.keras.Model.fit` method and passing it the appropriate hyperparameters.

In [ ]:
# Fit the model to a set of data
model.fit(x=X_train, # data to find patterns in 
          y=y_train, # labels for x
          epochs=1, # how many times to go over the whole dataset
          batch_size=32, # number of samples to look at each time
          validation_data=(X_val, # data to validate the learned patterns
                           y_val)) 

### Evaluate Model
Once you’ve trained a model, you can evaluate its performance on unseen data with `tf.keras.Model.evaluate`:

In [ ]:
# Evaluate a model on unseen data
loss, accuracy = model.evaluate(x=X_test, 
                                y=y_test)

### Save and Load a Whole TensorFlow Model
There are two main ways to save a whole TensorFlow model:

- **`SavedModel` format** — default format for saving a model in TensorFlow, saves a complete TensorFlow program including trained parameters and does not require the original model building code to run. Useful for sharing models across TensorFlow.js, TFLite, TensorFlow Serving and more.

- **`HDF5` format** — a more widely used data standard, however, does not contain as much information as the `SavedModel` format.


You can read the pros and cons of each of these in the TensorFlow documentation on Save and load Keras models.

In [ ]:
# Save model to SavedModel format (default)
model.save('model_save_path')

# Load model back in from SavedModel format
loaded_model = tf.keras.models.load_model('model_save_path')